In [ ]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 697.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
from groq import Groq

In [ ]:
def draft_message(content, role = 'user'):
  return {
      "role": role,
      "content": content,

  }
def truncate_content(content, max_length=4000):
    return content[:max_length]

In [ ]:
api_key = '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'

In [ ]:
client = Groq(api_key = api_key)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Dailyhunt_dataset/Dailyhunt_telugu.csv')
df.head()
news_text = list(df['Content'])

'తెలుగు సినీ పరిశ్రమలో అద్భుతమైన గుర్తింపును సంపాదించుకున్న దర్శకులలో ఒకరు అయినటువంటి వైవిఎస్ చౌదరి ఆఖరుగా సాయి ధరమ్ తేజ్ హీరోగా రూపొందిన రేయ్ అనే సినిమాకు దర్శకత్వం వహించాడు.\nఈ సినిమా పర్వాలేదు అనే రేంజ్ అంచనాల నడుమ థియేటర్లలో విడుదల అయ్యి బాక్స్ ఆఫీస్ దగ్గర ఘోర పరాజయాన్ని ఎదుర్కొంది. ఈ మూవీ విడుదల అయ్యి ఇప్పటికే చాలా సంవత్సరాలు అవుతోంది. ఇప్పటివరకు ఈ దర్శకుడు ఏ మూవీ కూడా చేయలేదు. ఇకపోతే కొన్ని రోజుల క్రితమే ఆయన తన తదుపరి మూవీ కి సంబంధించిన వివరాలను వెల్లడించాడు.\n\nఆ కార్యక్రమంలో భాగంగా ఈయన వైవిఎస్ చౌదరి దర్శకత్వం వహించిన దేవదాసు సినిమా సమయంలో జరిగిన కొన్ని సంఘటనల గురించి వివరించారు. తాజాగా ఈయన మాట్లాడుతూ ... నేను రామ్ హీరోగా ఇలియానా హీరోయిన్గా దేవదాసు సినిమాను చేశాను. ఆ సినిమా సమయంలో రామ్ కి ఎలాంటి క్రేజ్ లేదు. నేను ఆ సినిమాని సంక్రాంతి కానుకగా జనవరి 11 వ తేదీన విడుదల చేశాను. సంక్రాంతి సీజన్ లో పెద్దగా లవ్ స్టోరీ సినిమాలను విడుదల చేయరు.\n\nసంక్రాంతి సీజన్ అంతా ఫ్యామిలీ సినిమాలని జనాలు ఎక్కువ చూస్తుంటారు. అయినా నేను రిస్కు చేసి ఆ మూవీ ని విడుదల చేశాను. ఇక నా మూవీజనవరి 11.వ తేదీన విడ

In [ ]:
from tqdm import tqdm
prompt = ''' Craft a precise and engaging news headline in same language as of the news content that primarily summarizes the main point of the article. Follow these guidelines:

1. Focus predominantly (about 80-90%) on the key information and main message from the article.
2. Keep the headline concise, ideally under 15 words.
3. Use active voice and clear, impactful language.
4. Ensure the headline is factual and directly related to the article's main content.
'''

In [ ]:
df.to_csv('/content/drive/MyDrive/Dailyhunt_dataset/Dailyhunt_malayalam.csv', index=False)

In [ ]:
from groq import Groq
import pandas as pd
# from google.colab import drive
from tqdm import tqdm

def draft_message(content, role='user'):
    return {
        "role": role,
        "content": content,
    }

def truncate_content(content, max_length=4500):
    return content[:max_length]

api_key = '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'
client = Groq(api_key=api_key)

# drive.mount('/content/drive')


df = pd.read_csv('/content/drive/MyDrive/Dailyhunt_dataset/Dailyhunt_telugu.csv')

generated_headlines = []

for i in tqdm(range(len(news_text)), desc="Generating Headlines"):
    content = news_text[i]
    messages = [
        {
            "role": 'system',
            "content": f'''news: {content}'''
        }
    ]
    messages.append(draft_message(prompt))

    try:
        chat_completion = client.chat.completions.create(
            temperature=1.0,
            n=1,
            model='llama3-8b-8192',
            max_tokens=2000,
            messages=messages
        )
        generated_headline = chat_completion.choices[0].message.content
    except Exception as e:
        error_message = str(e)
        if "context_length_exceeded" in error_message:
            print(f"Error generating headline for article {i+1}: Context length exceeded. Truncating content.")
            truncated_content = truncate_content(content)
            messages[0]['content'] = f'''news: {truncated_content}'''
            try:
                chat_completion = client.chat.completions.create(
                    temperature=1.0,
                    n=1,
                    model='llama3-8b-8192',
                    max_tokens=2000,
                    messages=messages
                )
                generated_headline = chat_completion.choices[0].message.content
            except Exception as e2:
                print(f"Error generating headline for article {i+1} after truncation: {e2}")
                generated_headline = "Error generating headline"
        else:
            print(f"Error generating headline for article {i+1}: {e}")
            generated_headline = "Error generating headline"

    generated_headlines.append(generated_headline)

df['Generated_Headline'] = generated_headlines

df.to_csv('/content/drive/MyDrive/Dailyhunt_dataset/Dailyhunt_telugu.csv', index=False)

print("done sal!")


Generating Headlines:   9%|▊         | 43/505 [03:22<49:39,  6.45s/it]

Error generating headline for article 44: Context length exceeded. Truncating content.


Generating Headlines:   9%|▉         | 46/505 [04:01<1:20:24, 10.51s/it]

Error generating headline for article 47: Context length exceeded. Truncating content.


Generating Headlines:  11%|█         | 56/505 [05:37<1:08:03,  9.10s/it]

...

Error generating headline for article 446: Context length exceeded. Truncating content.


Generating Headlines:  99%|█████████▊| 498/505 [59:05<00:43,  6.21s/it]

Error generating headline for article 499: Context length exceeded. Truncating content.


Generating Headlines: 100%|██████████| 505/505 [59:53<00:00,  7.12s/it]

done sal!
